In [3]:
%matplotlib inline

In [4]:
import requests
import re
from fastai.text import *
import torch

In [5]:
book = requests.get('https://www.gutenberg.org/files/36/36-0.txt').text

In [6]:
start_iter = re.finditer('BOOK ONE', book)

In [7]:
next(start_iter)

<_sre.SRE_Match object; span=(1067, 1075), match='BOOK ONE'>

Commenting out some print lines here to shrink the notebook -- I did some exploration to figure out where the actual story began and ended (excluding table of contents, acknowledgements, etc.)

In [8]:
# book[1067:]

In [9]:
next(start_iter)

<_sre.SRE_Match object; span=(1920, 1928), match='BOOK ONE'>

In [10]:
# book[1920:]

In [11]:
book = book[1920:]

In [12]:
end_iter = re.finditer("among the dead.", book)

In [13]:
next(end_iter)

<_sre.SRE_Match object; span=(64109, 64124), match='among the dead '>

In [14]:
# book[:64125]

In [15]:
next(end_iter)

<_sre.SRE_Match object; span=(345594, 345609), match='among the dead.'>

In [16]:
# book[:345609]

In [17]:
book = book[:345609]

In [18]:
book = re.sub(r'[\r\n]+', ' ', str(book))

In [19]:
# book

In [20]:
tokens = [i.replace(',', '') for i in book.split('.')]

In [22]:
# tokens

In [23]:
tokens[:10]

['BOOK ONE THE COMING OF THE MARTIANS I',
 ' THE EVE OF THE WAR',
 ' No one would have believed in the last years of the nineteenth century that this world was being watched keenly and closely by intelligences greater than manâ\x80\x99s and yet as mortal as his own; that as men busied themselves about their various concerns they were scrutinised and studied perhaps almost as narrowly as a man with a microscope might scrutinise the transient creatures that swarm and multiply in a drop of water',
 ' With infinite complacency men went to and fro over this globe about their little affairs serene in their assurance of their empire over matter',
 ' It is possible that the infusoria under the microscope do the same',
 ' No one gave a thought to the older worlds of space as sources of human danger or thought of them only to dismiss the idea of life upon them as impossible or improbable',
 ' It is curious to recall some of the mental habits of those departed days',
 ' At most terrestrial men fa

In [24]:
! rm book_text.csv

In [25]:
with open('book_text.csv', 'a+') as file:
    for sentence in tokens:
        file.write(sentence + '\n')

In [26]:
with open('book_text.csv') as file:
    lines = file.readlines()[:5]
print(lines[0])
print(lines[1])
print(lines[2])

BOOK ONE THE COMING OF THE MARTIANS I

 THE EVE OF THE WAR

 No one would have believed in the last years of the nineteenth century that this world was being watched keenly and closely by intelligences greater than manâs and yet as mortal as his own; that as men busied themselves about their various concerns they were scrutinised and studied perhaps almost as narrowly as a man with a microscope might scrutinise the transient creatures that swarm and multiply in a drop of water



In [27]:
data = TextLMDataBunch.from_csv('.', 'book_text.csv', text_cols=0, max_vocab=1000, min_freq=2)

In [28]:
data

TextLMDataBunch;

Train: LabelList (2434 items)
x: LMTextList
xxbos i thought at once that it would xxunk my xxunk from the xxunk of the blow i had xxunk him,xxbos xxmaj the xxunk of xxunk xxunk xxunk is so xxunk xxunk in the xxmaj xxunk mind and xxunk intelligence so much a matter of course in the papers that they could read without any xxunk xxunk : xxunk xxunk oâclock last night the xxmaj martians came out of the cylinder and moving about under an xxunk of metallic xxunk have xxunk xxunk xxmaj woking station with the xxunk houses and xxunk an xxunk xxunk of the xxmaj xxunk xxmaj xxunk,xxbos xxmaj among these were a couple of xxunk a xxunk xxunk i xxunk xxunk a xxunk carrying a xxunk xxmaj xxunk the xxunk and his little boy and two or three xxunk and xxunk xxunk who were xxunk to xxunk about the railway station,xxbos xxmaj in the xxunk of bright xxunk sunlight i saw the xxmaj martian in its xxmaj xxunk of a handling - machine xxunk the xxunk head,xxbos xxmaj it may be in my time xx

In [29]:
data.bptt = 3

In [30]:
data.train_ds[0]

(Text xxbos i thought at once that it would xxunk my xxunk from the xxunk of the blow i had xxunk him,
 EmptyLabel )

In [31]:
data.train_ds[1][0].data

array([2, 5, 9, 0, ..., 5, 0, 5, 0])

In [32]:
data.valid_ds[1][0].data

array([  2,   5,  24, 253, 171,  14, 378,   9,   0])

In [33]:
data.one_batch()[0].shape

torch.Size([64, 70])

In [34]:
x, y = data.one_batch()

In [35]:
x.shape, y.shape

(torch.Size([64, 70]), torch.Size([64, 70]))

In [36]:
x[0], y[0]

(tensor([  0,  14,   0,  50,   9, 284, 193,   2,   5,  15,   9,   0,  11, 475,
           0, 685,  13,  71,   9,   5, 103,  15,  58,   5,   0,  11,  12, 395,
          29, 165,   0,   9,   0, 285,   2,   5,  18, 194,  62,  15,  21,  72,
           0,  78,   2,   5,  10,  51,  12, 166, 195, 762,   2,   5, 240,  52,
          15,   5,   0,   5, 622,  10,   5,   0, 133,  42, 119,   9, 864,  11]),
 tensor([ 14,   0,  50,   9, 284, 193,   2,   5,  15,   9,   0,  11, 475,   0,
         685,  13,  71,   9,   5, 103,  15,  58,   5,   0,  11,  12, 395,  29,
         165,   0,   9,   0, 285,   2,   5,  18, 194,  62,  15,  21,  72,   0,
          78,   2,   5,  10,  51,  12, 166, 195, 762,   2,   5, 240,  52,  15,
           5,   0,   5, 622,  10,   5,   0, 133,  42, 119,   9, 864,  11,  18]))

In [37]:
len(data.vocab.itos)

1006

In [38]:
nv = len(data.vocab.itos)

In [39]:
nh = 64

In [40]:
bs = 64

In [41]:
class BasicLanguageModel(nn.Module):
     
    def __init__(self):
        super().__init__()
        self.i_h = nn.Embedding(nv, nh)
        self.h_h = nn.Linear(nh, nh)
        self.h_o = nn.Linear(nh, nv)
        self.bn = nn.BatchNorm1d(nh)
        self.h = torch.zeros(bs, nh).cuda()
        
    def forward(self, x):
        res = []
        h = self.h
        for i in range(x.shape[1]):
            h = h + self.i_h(x[:,i]) 
            h = F.relu(self.h_h(h))
            res.append(self.bn(h))
        self.h = h.detach()
        res = torch.stack(res, dim=1)
        res = self.h_o(res)
        return res

In [42]:
data.label_list

LabelLists;

Train: LabelList (2434 items)
x: LMTextList
xxbos i thought at once that it would xxunk my xxunk from the xxunk of the blow i had xxunk him,xxbos xxmaj the xxunk of xxunk xxunk xxunk is so xxunk xxunk in the xxmaj xxunk mind and xxunk intelligence so much a matter of course in the papers that they could read without any xxunk xxunk : xxunk xxunk oâclock last night the xxmaj martians came out of the cylinder and moving about under an xxunk of metallic xxunk have xxunk xxunk xxmaj woking station with the xxunk houses and xxunk an xxunk xxunk of the xxmaj xxunk xxmaj xxunk,xxbos xxmaj among these were a couple of xxunk a xxunk xxunk i xxunk xxunk a xxunk carrying a xxunk xxmaj xxunk the xxunk and his little boy and two or three xxunk and xxunk xxunk who were xxunk to xxunk about the railway station,xxbos xxmaj in the xxunk of bright xxunk sunlight i saw the xxmaj martian in its xxmaj xxunk of a handling - machine xxunk the xxunk head,xxbos xxmaj it may be in my time xxunk m

In [43]:
data.bptt = 20
data.batch_size = bs

In [44]:
learn = Learner(data, BasicLanguageModel(), metrics=accuracy)

In [45]:
learn.fit_one_cycle(20, max_lr=3e-2)

epoch,train_loss,valid_loss,accuracy,time
0,6.831203,6.661967,0.102734,00:00
1,6.572461,5.902810,0.190960,00:00
2,5.965570,4.373462,0.227846,00:00
3,5.339042,4.045414,0.254743,00:00
4,4.893654,3.925138,0.271875,00:00
5,4.564156,3.883368,0.273382,00:00
6,4.311296,3.885398,0.278125,00:00
7,4.112661,3.891879,0.274386,00:00
8,3.947726,3.917398,0.280301,00:00
9,3.807902,3.918825,0.280078,00:00


# TODO: try a GRU/LSTM